In [2]:
import torch
import numpy as np

[HAMI-core Msg(48810:139736112688000:libvgpu.c:836)]: Initializing.....


In [3]:
torch.__version__

'2.4.0+cu121'

### 基本数据处理与计算操作
#### 创建Tensor

In [28]:
# 创建一个2x3的未初始化的Tensor
x1 = torch.empty(2, 3)
# 创建一个2x3的随机初始化的Tensor
x2 = torch.rand(2,3)
# 创建一个2x3的long型全0的Tensor
x3 = torch.zeros(2, 3, dtype=torch.long)
# 为每一个元素以给定的mean和std用高斯分布生成随机数
x4 = torch.normal(mean=0.5, std=torch.arange(1., 6.))
# randn 服从N(0，1)的正态分布
x5 = torch.randn(2, 3)

x1, x2, x3, x4, x5

(tensor([[1.0825e-34, 0.0000e+00, 0.0000e+00],
         [1.4013e-45, 0.0000e+00, 0.0000e+00]]),
 tensor([[0.9814, 0.0874, 0.5718],
         [0.6293, 0.1877, 0.5226]]),
 tensor([[0, 0, 0],
         [0, 0, 0]]),
 tensor([ 1.5466, -3.7387, -2.4466,  3.9320,  0.3221]),
 tensor([[-1.7507, -0.7708, -0.1977],
         [ 1.7947,  2.4549, -0.1717]]))

In [32]:
# 根据不同数据类型创建
x1 = torch.tensor([[5,5,3], [2,2,5]])
x2 = torch.tensor(np.array([[1, 2, 3], [4, 5, 6]]))
x3 = torch.from_numpy(np.array([[1, 2, 3], [4, 5, 6]]))
x1, x2, x3

(tensor([[5, 5, 3],
         [2, 2, 5]]),
 tensor([[1, 2, 3],
         [4, 5, 6]]),
 tensor([[1, 2, 3],
         [4, 5, 6]]))

In [35]:
x = torch.tensor([[1,2,3], [4,5,6]])
# 返回的tensor默认具有相同的torch.dtype和torch.device 
x1 = x.new_ones(2, 3, dtype=torch.float64)
# 指定新的数据类型
x2 = torch.randn_like(x, dtype=torch.float, device='cuda')
x1, x2

(tensor([[1., 1., 1.],
         [1., 1., 1.]], dtype=torch.float64),
 tensor([[ 1.2797, -0.0880,  1.2151],
         [ 1.1600, -0.2267,  0.4583]], device='cuda:0'))

#### 操作张量

##### 算术操作

In [43]:
x = torch.ones(2, 3) 
y = torch.eye(2, 3)
# 加法形式1
s1 = x + y
# 加法形式2
s2 = torch.add(x, y)
# 加法形式3，inplace(原地操作)，原值修改
s3 = y.add_(x)

x, y, s1, s2, s3, y

(tensor([[1., 1., 1.],
         [1., 1., 1.]]),
 tensor([[2., 1., 1.],
         [1., 2., 1.]]),
 tensor([[2., 1., 1.],
         [1., 2., 1.]]),
 tensor([[2., 1., 1.],
         [1., 2., 1.]]),
 tensor([[2., 1., 1.],
         [1., 2., 1.]]),
 tensor([[2., 1., 1.],
         [1., 2., 1.]]))

##### 索引
索引出来的结果与原数据`共享内存`，也即修改一个，另一个会跟着修改。

In [45]:
x = torch.ones(2, 3, device='cuda') 
y = x[0, :]
y += 1

x, y, x[0, :]

(tensor([[2., 2., 2.],
         [1., 1., 1.]], device='cuda:0'),
 tensor([2., 2., 2.], device='cuda:0'),
 tensor([2., 2., 2.], device='cuda:0'))

##### 切片 index_select
沿着指定维度对输入进行切片，取index中指定的相应项(index 为一个 LongTensor)，然后返回到一个新的张量， 返回的张量与原始张量 Tensor 有相同的维度(在指定轴上)。

注意： `返回的张量不与原始张量共享内存空间`。<br>

参数:<br>
input (Tensor)         – 输入张量<br>
dim (int)              – 索引的轴<br>
index (LongTensor)     – 包含索引下标的一维张量<br>
out (Tensor, optional) – 目标张量<br>

In [46]:
x = torch.randn(3, 4)
indices = torch.LongTensor([0, 2])
y = torch.index_select(x, 0, indices)
x, indices, y

(tensor([[ 0.1912,  0.1199, -0.0856, -1.8215],
         [ 0.2932,  0.9630,  0.3461, -0.7105],
         [-1.2408, -1.6029,  0.8402,  0.3389]]),
 tensor([0, 2]),
 tensor([[ 0.1912,  0.1199, -0.0856, -1.8215],
         [-1.2408, -1.6029,  0.8402,  0.3389]]))

##### 改变形状 view/reshape
1. `view()`<br>
注意view()**返回的新Tensor与源Tensor虽然可能有不同的size**，但`共享data`。<br>
即更改其中的一个，另外一个也会跟着改变。(顾名思义，view仅仅是改变了对这个张量的 观察角度，内部数据并未改变)
2. `reshape()` 和 `clone()`<br> 
Pytorch还提供了一个 reshape() 方法可以改变形状，但是此函数并不能保证返回的是其拷贝，所以不推荐使用。<br> 
我们推荐先用 clone() 创造一个副本然后再使用view()。

In [51]:
x = torch.ones(2, 4)
# 一个tensor必须是连续的contiguous()才能被查看。
# 一开始不加contiguous()，报 “view size is not compatible ... ” 错误
y1 = x.view(8)
y2 = x.view(-1, 8) # -1所指的维度可以根据其他维度的值推出来

x, y1, y2, x.size(), y1.size(), y2.size()

(tensor([[1., 1., 1., 1.],
         [1., 1., 1., 1.]]),
 tensor([1., 1., 1., 1., 1., 1., 1., 1.]),
 tensor([[1., 1., 1., 1., 1., 1., 1., 1.]]),
 torch.Size([2, 4]),
 torch.Size([8]),
 torch.Size([1, 8]))

In [52]:
x += 1
print(x)
print(y1) # 也加了1

tensor([[2., 2., 2., 2.],
        [2., 2., 2., 2.]])
tensor([2., 2., 2., 2., 2., 2., 2., 2.])


In [53]:
# torch.view()会改变原始张量，但是很多情况下，我们希望原始张量和变换后的张量互相不影响。为为了使创建的张量和原始张量不共享内存，我们需要使用第二种方法torch.reshape()， 同样可以改变张量的形状，但是此函数并不能保证返回的是其拷贝值，所以官方不推荐使用。推荐的方法是我们先用 `clone()` 创造一个张量副本然后再使用 `torch.view()`进行函数维度变换。<br>
# 注：使用 clone() 还有一个好处是会被记录在计算图中，即梯度回传到副本时也会传到源 Tensor 。 

x1 = x.clone()
y3 = x.view(-1)
x1 += 1

x1, y3

(tensor([[3., 3., 3., 3.],
         [3., 3., 3., 3.]]),
 tensor([2., 2., 2., 2., 2., 2., 2., 2.]))

##### gather
`torch.gather(input, dim, index, out=None)`<br>
官方定义：沿给定轴dim，将输入索引张量index指定位置的值进行聚合。<br>
通俗理解：给定轴dim，在input中，根据index指定的下标，选择元素重组成一个新的tensor，最后输出的out与index的size是一样的。<br>

对一个3维张量，输出可以定义为<br>

```
out[i][j][k] = tensor[index[i][j][k]][j][k]] # dim=0
out[i][j][k] = tensor[i][index[i][j][k]][k]] # dim=1
out[i][j][k] = tensor[i][j][index[i][j][k]]] # dim=3
```

In [56]:
_input = torch.tensor([[0.9, 0, 0], [0, 0.6, 0],[0, 0.7, 0]])
index = torch.tensor([0, 1, 1])
_input, index, index.view(-1, 1)

(tensor([[0.9000, 0.0000, 0.0000],
         [0.0000, 0.6000, 0.0000],
         [0.0000, 0.7000, 0.0000]]),
 tensor([0, 1, 1]),
 tensor([[0],
         [1],
         [1]]))

In [30]:
_input.gather(dim=1, index=index.view(-1, 1))
# dim=1，表示的是在第二维度上操作。
# 在index中，[0]表示第一行对应元素的下标，即[0.9]
#           [1]表示第二行对应元素的下标，即[0.6]
#           [1]表示第三行对应元素的下标，即[0.7]


tensor([[0.9000],
        [0.6000],
        [0.7000]])

#### 广播机制
当我们对两个形状不同的Tensor按元素运算时，可能会触发`广播(broadcasting)机制`。 <br>
先适当复制元素使这两个Tensor形状相同后再按元素运算。

In [57]:
# x 中第一行的2个 元素被广播(复制)到了第二行和第三行
# y 中第一列的3个元素被广播(复制)到 了第二列
x = torch.arange(1, 3).view(1, 2)
y = torch.arange(1, 4).view(3, 1)
x, y, x+y

(tensor([[1, 2]]),
 tensor([[1],
         [2],
         [3]]),
 tensor([[2, 3],
         [3, 4],
         [4, 5]]))

#### Tensor和NumPy相互转换
+ `numpy()`和`from_numpy()` <br>
这两个函数所产生的的Tensor和NumPy中的数组共享相同的内存(所以他们之间的转换很快)，改变其中一个时另一个也会改变!
+ `torch.tensor()` <br>
进行**数据拷贝**，所以返回的Tensor和原来 的数据不再共享内存。

In [63]:
# Tensor转NumPy数组
x = torch.ones(3)
y = x.numpy()
x += 1
y += 1
print(x, y)

# NumPy数组转 Tensor
x = np.ones(3)
y = torch.from_numpy(x)
x += 1
y += 1
print(x, y)

tensor([3., 3., 3.]) [3. 3. 3.]
[3. 3. 3.] tensor([3., 3., 3.], dtype=torch.float64)


In [65]:
# 使用torch.tensor()将NumPy数组转换成Tensor(不再共享内存)
x = np.ones(3)
y = torch.tensor(x)
x += 1
print(x, y)

[2. 2. 2.] tensor([1., 1., 1.], dtype=torch.float64)


#### 自动求梯度（Autograd）
torch.Tensor 是这个包的核心类。如果设置它的属性 `.requires_grad` 为 True，那么它将会追踪对于该张量的所有操作。
当完成计算后可以通过调用 `.backward()`，来自动计算所有的梯度。这个张量的所有梯度将会自动累加到`.grad`属性。<br>

如果不想要被继续追踪，可以调用`.detach()`将其从追踪记录中分离出来，这样就可以防止将来的计算被追踪。
此外，还可以用`with torch.no_grad()`将不想被追踪的操作代码块包裹起来，这种方法在评估模型的时候很常用，因为在评估模型时，我们并不需要计算 可训练参数(requires_grad=True)的梯度。<br>

Tensor和Function互相结合就可以构建一个记录有整个计算过程的**有向无环图(DAG)**。
每个Tensor都有一个`.grad_fn`属性，用来记录创建张量时所用到的运算，在链式求导法则中会使用到，默认是None。<br>

+ 自动求导机制通过有向无环图（directed acyclic graph ，DAG）实现
+ 在DAG中，记录数据（对应tensor.data）以及操作（对应tensor.grad_fn）
+ 操作在pytorch中统称为`Function`，如加法、减法、乘法、ReLU、conv、Pooling等

In [85]:
# 标量求梯度
w = torch.tensor([1.], requires_grad=True)
x = torch.tensor([2.], requires_grad=True)
a = torch.add(w, x)
b = torch.add(w, 1)
y = torch.mul(a, b)

 # x是直接创建的，所以它没有grad_fn
print(x.grad_fn)
print(a.grad_fn)

y.backward()
print(w.grad)

None
tensor([5.])


In [86]:
# 注意grad是累加的
y2 = w.sum()
y2.backward()      # 梯度未清零，累加梯度
print(w.grad)

y3 = w.sum()
w.grad.data.zero_()
y3.backward()      # 梯度清零后，x的梯度为1
print(w.grad)

tensor([6.])
tensor([1.])


In [87]:
from torch.autograd import Function

class Exp(Function):                    # 此层计算e^x
    @staticmethod
    def forward(ctx, i):                # 模型前向
        result = i.exp()
        ctx.save_for_backward(result)   # 保存所需内容，以备backward时使用，所需的结果会被保存在saved_tensors元组中；
                                        # 此处仅能保存tensor类型变量，若其余类型变量（Int等），可直接赋予ctx作为成员变量，也可以达到保存效果
        return result
    @staticmethod
    def backward(ctx, grad_output):     # 模型梯度反传
        result, = ctx.saved_tensors     # 取出forward中保存的result
        return grad_output * result     # 计算梯度并返回


# 尝试使用
x = torch.tensor([1.], requires_grad=True)  # 需要设置tensor的requires_grad属性为True，才会进行梯度反传
ret = Exp.apply(x)                          # 使用apply方法调用自定义autograd function
print(ret)                                  # tensor([2.7183], grad_fn=<ExpBackward>)
ret.backward()                              # 反传梯度
print(x.grad)                               # tensor([2.7183])

tensor([2.7183], grad_fn=<ExpBackward>)
tensor([2.7183])
